In [10]:
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import paddle
def synthetic_data(w, b, num_examples):
    """生成y=Xw+b+噪声"""
    X = paddle.normal(0, 1, (num_examples, len(w)))
    y = paddle.matmul(X, w) + b
    y += paddle.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = paddle.to_tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print(len(true_w))

2


In [11]:

def load_array(data_arrays, batch_size, is_train=True):
    """构造一个Paddle数据迭代器"""
    dataset = paddle.io.TensorDataset(data_arrays)
    return paddle.io.DataLoader(dataset, batch_size=batch_size,
                                shuffle=is_train,
                                return_list=True)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[Tensor(shape=[10, 2], dtype=float32, place=Place(gpu:0), stop_gradient=True,
        [[-0.21706608, -0.05333515],
         [-1.15808201, -0.00936312],
         [ 0.64921069, -1.29995191],
         [ 1.18909514, -0.51112437],
         [-0.52607369,  0.23117609],
         [-0.50837523, -0.37801442],
         [ 2.02070999, -0.59552199],
         [-0.78526264, -2.53457332],
         [-0.60161507, -0.75507200],
         [ 1.21321237,  0.71976131]]),
 Tensor(shape=[10, 1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
        [[3.92788410 ],
         [1.90915704 ],
         [9.89881134 ],
         [8.29380512 ],
         [2.36448121 ],
         [4.47440910 ],
         [10.27638340],
         [11.24937153],
         [5.57783747 ],
         [4.18148613 ]])]

In [13]:
# nn是神经网络的缩写
from paddle import nn

net = nn.Sequential(nn.Linear(2, 1))

In [ ]:
wei_attr = paddle.ParamAttr(initializer=paddle.nn.initializer.Normal(0, 0.01))
bias_attr = paddle.ParamAttr(initializer=None)